## 1. IMPORTATION DES BIBLIOTHÈQUES NÉCESSAIRES

In [1]:
import pandas as pd
import numpy as np
import joblib

# Pour la modélisation et la préparation des données
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Pour l'évaluation des performances
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

## 2. Chargement des données

In [2]:
print("--- 1. Configuration des chemins de fichiers ---")
OUTPUT_PREDICTIONS_FILE = 'predictions_test.csv' 
MODEL_ARTIFACT = 'modele_diabete_XX.pkl'


--- 1. Configuration des chemins de fichiers ---


In [3]:
print("\n--- 2. Chargement des ensembles Train et Test ---")

# Chargement du fichier d'entraînement (déjà nettoyé et encodé)
try:
    df_train = pd.read_csv('data/diabetes_clean.csv')
    df_test_raw = pd.read_csv('data/test_without_class.csv')
except FileNotFoundError as e:
    print(f"ERREUR : Fichier non trouvé - {e.filename}")
    exit()

## --- Séparation X et y pour l'ENTRAINEMENT ---
X_train_full = df_train.drop('class', axis=1)
y_train_full = df_train['class']

# --- Préparation de l'ensemble de TEST (X_test) ---
test_ids = df_test_raw['ID'] 
X_test = df_test_raw.drop('ID', axis=1).copy() # X_test doit être une copie pour les modifications

# Standardisation des noms de colonnes pour X_test (crucial pour l'alignement)
X_test.columns = [col.lower().replace(' ', '_') for col in X_test.columns] 


# --- ENCODAGE DU FICHIER TEST EN BINAIRE (LA NOUVELLE ÉTAPE) ---
print("-> Encodage binaire (0/1) des colonnes textuelles du fichier de test...")

le = LabelEncoder()

for col in X_test.columns:
    if X_test[col].dtype == 'object':
        
        # Pour garantir le même mapping (ex: No->0, Yes->1), nous créons un mapping manuel
        # basé sur les valeurs probables du diabète (crucial pour l'interprétabilité)
        if col == 'gender':
            # Le genre a un mapping inversé dans l'EDA (Male=1, Female=0)
            mapping = {'Female': 0, 'Male': 1}
        else:
            # Tous les autres symptômes ont un mapping No=0, Yes=1
            mapping = {'No': 0, 'Yes': 1}
            
        X_test[col] = X_test[col].map(mapping)
        
        # Remplacer les NaN (si jamais une valeur inattendue apparaissait) par 0
        X_test[col] = X_test[col].fillna(0).astype(int)

print(f"Aperçu des features encodées de Test (doivent être numériques) : \n{X_test.head()}")



--- 2. Chargement des ensembles Train et Test ---
-> Encodage binaire (0/1) des colonnes textuelles du fichier de test...
Aperçu des features encodées de Test (doivent être numériques) : 
   age  gender  polyuria  polydipsia  sudden_weight_loss  weakness  \
0   50       0         0           0                   0         1   
1   55       1         0           1                   0         1   
2   67       1         1           1                   0         1   
3   45       1         0           0                   0         0   
4   37       1         0           0                   0         0   

   polyphagia  genital_thrush  visual_blurring  itching  irritability  \
0           0               0                1        1             0   
1           0               1                0        0             1   
2           1               1                0        1             1   
3           1               1                0        0             0   
4           0            

## 3. PIPELINE DE PRÉTRAITEMENT ET ENTRAÎNEMENT FINAL

In [4]:
print("\n--- 3. Création et Entraînement du Pipeline Final ---")

# Définition de la standardisation
numerical_features = ['age']
preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)

# Modèle de classification choisi (Random Forest)
best_model = RandomForestClassifier(
    n_estimators=100, 
    random_state=42, 
    class_weight='balanced'
)

# Création du pipeline complet
final_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', best_model)
])

# Entraînement du pipeline sur TOUTES les données d'entraînement disponibles
final_pipeline.fit(X_train_full, y_train_full) 
print("Entraînement final sur l'ensemble complet terminé.")


--- 3. Création et Entraînement du Pipeline Final ---
Entraînement final sur l'ensemble complet terminé.


## 4. PRÉTRAITEMENT (SCALING) - UNIQUEMENT SUR LA COLONNE 'AGE'

In [5]:
print("\n--- 4. Prédictions et Création du Fichier de Soumission ---")

# Prédictions de classe. Le modèle prédit déjà 0 (Négatif) ou 1 (Positif).
predictions_binary = final_pipeline.predict(X_test) 

# L'étape de reconversion en labels (Negative/Positive) est supprimée pour garder le format binaire.

# Création du DataFrame de soumission (ID et class)
submission_df = pd.DataFrame({
    'ID': test_ids,
    # Utilisation directe des prédictions binaires (0/1)
    'class': predictions_binary 
})

# Sauvegarde du fichier
submission_df.to_csv(OUTPUT_PREDICTIONS_FILE, index=False)

print(f"Fichier de soumission créé avec succès : '{OUTPUT_PREDICTIONS_FILE}' avec la colonne 'class' en binaire (0/1).")


# --- SAUVEGARDE DU MODÈLE (LIVRABLE .pkl) ---
joblib.dump(final_pipeline, MODEL_ARTIFACT)
print(f"Artefact modèle sauvegardé : '{MODEL_ARTIFACT}'")
print("\n--- FIN DU SCRIPT ---")


--- 4. Prédictions et Création du Fichier de Soumission ---
Fichier de soumission créé avec succès : 'predictions_test.csv' avec la colonne 'class' en binaire (0/1).
Artefact modèle sauvegardé : 'modele_diabete_XX.pkl'

--- FIN DU SCRIPT ---
